<a href="https://colab.research.google.com/github/abhinav4532raj/resume_builder/blob/main/resume_builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn spacy flask-ngrok
!python -m spacy download en_core_web_sm
!pip install pyngrok




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
from pyngrok import ngrok

# Replace 'your_auth_token' with the auth token you copied
ngrok.set_auth_token("2tM2YaCxD3PC310oSvT1Zrdx0d7_4LckvtaKXFfbANwVMenax")


# Replace 'your_auth_token' with the auth token you copied



# New Section

In [9]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from flask import Flask, request, jsonify
from pyngrok import ngrok
from google.colab import files


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def extract_skills(text):
    doc = nlp(text)
    skills = [ent.text for ent in doc.ents if ent.label_ == 'SKILL']
    return skills

data['skills'] = data['processed_text'].apply(extract_skills)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Vectorize job descriptions
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['processed_text'])

def recommend_jobs(user_resume):
    user_tfidf = vectorizer.transform([user_resume])
    cosine_similarities = linear_kernel(user_tfidf, tfidf_matrix).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-5:-1]
    return data.iloc[related_docs_indices]

user_resume = preprocess_text("User resume text here")
recommendations = recommend_jobs(user_resume)
recommendations


,ID,Resume_str,Resume_html,Category,processed_text,skills
1536,15011085,DIRECTOR OF FINANCE Summary ...,"<div class=""fontsize fontface vmargins hmargin...",FINANCE,15011085 director of finance summary senior fi...,[]
1902,35554162,ACCOUNTANT III Senior Accountan...,"<div class=""fontsize fontface vmargins hmargin...",ACCOUNTANT,35554162 accountant iii senior accountant resu...,[]
1856,18132924,ACCOUNTANT Executive Profile ...,"<div class=""fontsize fontface vmargins hmargin...",ACCOUNTANT,18132924 accountant executive profile capable ...,[]
250,32959732,"SENIOR DIRECTOR, INFORMATION TECHNOLO...","<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,32959732 senior director information technolo...,[]


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import logging

# Set logging level
logging.basicConfig(level=logging.INFO)

# Load your dataset from a file
data = pd.read_csv('Resume.csv')

def preprocess_text(text):
    try:
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\W', ' ', text)
        text = text.lower()
        return text
    except Exception as e:
        logging.error(f"Error in preprocess_text: {e}")

# Ensure the correct column name is used
try:
    data['processed_text'] = data[['ID', 'Resume_str', 'Resume_html', 'Category']].apply(lambda x: preprocess_text(' '.join(x.astype(str))), axis=1)
    logging.info(f"Processed data:\n{data.head()}")
except Exception as e:
    logging.error(f"Error in data processing: {e}")

# Vectorize job descriptions
try:
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(data['processed_text'])
    logging.info(f"TFIDF matrix shape: {tfidf_matrix.shape}")
except Exception as e:
    logging.error(f"Error in vectorization: {e}")

def recommend_jobs(user_resume):
    try:
        user_tfidf = vectorizer.transform([user_resume])
        cosine_similarities = linear_kernel(user_tfidf, tfidf_matrix).flatten()
        related_docs_indices = cosine_similarities.argsort()[:-5:-1]
        recommendations = data.iloc[related_docs_indices]
        logging.info(f"Recommendations:\n{recommendations}")
        return recommendations
    except Exception as e:
        logging.error(f"Error in recommend_jobs: {e}")
        return None

# Set Ngrok auth token
ngrok.set_auth_token("2tM2YaCxD3PC310oSvT1Zrdx0d7_4LckvtaKXFfbANwVMenax")

# Create Flask app
app = Flask(__name__)

@app.route('/build_resume', methods=['POST'])
def build_resume():
    try:
        content = request.json
        logging.info(f"Received resume: {content['resume']}")
        user_resume = preprocess_text(content['resume'])
        recommended_jobs = recommend_jobs(user_resume)
        if recommended_jobs is None:
            raise ValueError("No recommendations found")
        return jsonify(recommended_jobs.to_dict()), 200
    except Exception as e:
        logging.error(f"Error in build_resume: {e}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print("Ngrok URL:", public_url)
    app.run()


Ngrok URL: NgrokTunnel: "https://dabf-34-32-148-27.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Feb/2025 16:08:42] "POST /build_resume HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2025 16:11:27] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Feb/2025 16:11:27] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    rais

In [26]:
!pip install flask-ngrok
!python app.py


<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/content/app.py", line 11, in <module>
    uploaded = files.upload()
               ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/files.py", line 72, in upload
    uploaded_files = _upload_files(multiple=True)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/files.py", line 164, in _upload_files
    result = _output.eval_js(
             ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/output/_js.py", line 36, in eval_js
    kernel = _ipython.get_kernel()
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_ipython.py", line 24, in get_kernel
    return get_ipython().kernel
           ^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'kernel'
